In [1]:
!pip install tensorflow==2.12.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 70.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.7 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.15.0
    Uninstalling wrapt-1.15.0:
      Successfully uninstalled wrapt-1.15.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.13.0
    Uninstalling tensorflow-estimator-2.13.0:
      Successfully uninstalled tensorflow-estimator-2.13.0
  Attempting uninstall: keras
    Found existing installation: keras 2.13.1
    Uninstalling keras-2.13.1:
      Successfully uninstalled keras-2.13.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.13.

In [1]:
import pandas as pd
import tensorflow as tf

df = pd.read_csv('/content/sample_data/profanity.csv')
df = df[['sentence', 'label']]
df = df.dropna()

df.head()

FileNotFoundError: ignored

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
shuffled_df = df.sample(frac=1, random_state=42).reset_index(drop=True)

In [5]:
from sklearn.model_selection import train_test_split
import numpy as np
x = df['sentence'].to_list()
y = df['label'].to_list()

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=.1, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=.2, random_state=42)


In [6]:
import tensorflow_hub as hub
from tensorflow.keras.layers import TextVectorization
max_vocab = 10000
max_length = 20

text_vectorizer = TextVectorization(max_tokens=max_vocab,
                                    output_mode="int",
                                    output_sequence_length=max_length)
text_vectorizer.adapt(x_train)

In [7]:
class CustomTextVectorization(tf.keras.layers.Layer):
    def __init__(self, max_tokens, output_sequence_length, **kwargs):
        super(CustomTextVectorization, self).__init__(**kwargs)
        self.max_tokens = max_tokens
        self.output_sequence_length = output_sequence_length
        self.text_vectorizer = tf.keras.layers.TextVectorization(
            max_tokens=max_tokens,
            output_mode="int",
            output_sequence_length=output_sequence_length,
        )

    def call(self, inputs):
        return self.text_vectorizer(inputs)

custom_text_vectorizer = CustomTextVectorization(max_tokens=max_vocab, output_sequence_length=max_length)

In [8]:
from tensorflow.keras import layers
Embedder = layers.Embedding(input_dim=max_vocab,
                                     output_dim=20,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding")


In [9]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = Embedder(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = tf.keras.Model(inputs, outputs, name="Model")

In [ ]:
model.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])


model_history = model.fit(x_train, y_train, epochs=100, validation_data=(x_val, y_val))

In [ ]:
def final_pred(quote):
  val = model.predict([quote])
  # Round
  rounded = np.round(val)

  # Convert to integer
  a_int = rounded.astype(int)
  return a_int[0][0]

print(final_pred(['I love your mom']))

In [ ]:
model.save('hate_speech.tf')

In [ ]:
!zip -r /content/hate_speech.tf.zip /content/hate_speech.tf
files.download('/content/hate_speech.tf.zip')